# 抽象表: ADT List 及其实现

可以认为 ADT 是一个抽象的数学概念. 我们定义它为一个由基本元素构成的集合,
以及相应的操作(operation). 比如我们可以定义一个数据类型名为 set,
一个 set 可以包含有限个或零个元素. 我们可以定义操作为 add, 即增加一个指定的元素；
操作 remove, 删除一个指定的元素；操作 size, 计算当前元素总数, 等等.

不同的编程语言, 都不难将上述概念具体化. 比如将元素具体为一个浮点数,
然后我们可以讨论如何实现一个有限的浮点数集, 并将其用于我们具体的问题中. 而 C++
提供了面向对象设计(object-oriented)的能力, 因此可以将这件事情做的更加直接. 一个
C++ 类(class), 允许将数据结构和操作捆绑在一起定义, 因此几乎可以直接视为一个 ADT
的实现.

## 抽象数据结构: 表(List)

我们称有限个元素组成的序列 $A_0$, $A_1$, $\cdots$, $A_{N - 1}$ 为一个 ADT List.
表的长度 (size) 为非负整数 $N$. 如果 $N == 0$ 为真, 我们称该表为空表(empty list).

对非空表中元素 $A_i$, $0 \leq i < N$, $A_{i - 1}$ 称为它的前任 (predecessor),
$A_{i + 1}$ 称为它的继任 (successor). 当然, 前任和继任可以不存在. 而自然数 $i$ 称为
$A_i$ 的位置 (position).


我们可以定义以下一些常见的操作(先尊重课本次序展开):
+ `printList` 输出表中全部元素. 空表则什么也不输出.
+ `makeEmpty` 清空列表, 删除全部元素. 空表则什么也不发生. 
+ `find (x)` 这里 `x` 是一个具体的值, 表中寻找值 `x`, 如果找到, 返回第一个找到的 `x` 的位置, 否则返回 NIL. 
+ `insert (x, pos)` 这里 `x` 是一个具体的值, `pos` 是一个位置, 即在指定位置 `pos` 前插入一个新元素 `x`, 插入之后新元素 `x` 在表中的位置将会是 `pos`. 如果 `pos` 位置不存在, 则报错.
+ `remove (x)` 删除 `x`, 如果表中没有 `x` 则什么事也不发生. 这里我们可以规定, 如果表中不止一个 `x`, 一次只删除第一个发现的那一个. 也即这里其实是先操作了一次 `find(x)`. 
+ `findKth (pos)` 返回第 `pos` 位置的元素. 如果位置 `pos` 不存在, 则报错.
+ `next (pos)` 返回第 `pos` 位置的继任元素的位置. 如果继任不存在, 则返回 NIL.
+ `previous (pos)` 返回第 `pos` 位置的前任元素的位置. 如果前任不存在, 则返回 NIL.

我们还可以根据需要, 定义和扩展表的操作功能. 而对于不同的具体实现, 这些功能定义也可能得不到严格的遵守(因此需要作出调整和改变).

在以上操作中, 我们发现有些操作是不会改变表的内容和结构的, 如`printList` 和 `find(x)`, 这些操作我们一般称为**静态操作**; 而有些操作则会改变, 如 `insert` 和 `remove`. 这里我们称其为**动态操作**.

## 表的简单数组(Array)实现

**数组(Array)** 是指用一段**连续**的内存空间来存放一个表. 由于表的每一个元素所占内存空间相等, 因此我们只需记录表的第一个元素所在内存地址, 那么就可以容易地获得数组的第 $k$ 个元素的内存地址. 在 C / C++ 中, 这个 `findKth` 的操作, 直接通过运算 `[]` 实现. 这种实现不但非常简单, 而且是计算机存储在系统层面上最基础的方式, 本质上, 我们的全部内存都可以看作按内存地址连续存放的一个大数组.

得益于数据在内存中的连续存放, 静态操作如 `findKth` 的时间代价一般为 $O(1)$, 而动态操作如 `insert` 和 `remove` 的代价是 $O(N)$. 这是因为我们在改变了表的结构之后, 必须确保数据仍然有序和连续存放. 

换言之, 用 Array 去实现 ADT List, 静态效率较高而动态效率较低. 

我们接下去用 C++ 实现这个操作, 主要目的是用这个简单的模型, 学习 C++ 的基本框架.

我们先考虑一个只能存放 `char` 类型的数组, 数组实现的关键是必须保持数据在内存中的连续性, 然后利用这一点加速静态操作. 但抛开这些具体的实现细节不管, 我们先实现我们的 List 的抽象概念定义: 

In [1]:
class ArrayList
{
private:
public:
    void printList();
    void makeEmpty();
    int find(char x);
    void insert(char x, int pos);
    void remove(char x);
    char findKth(int pos);
    int next(int pos);
    int previous(int pos);
};

写完这个定义我们就知道, `next` 和 `previous` 这两个操作是愚蠢的. 这里本质上我们唯一需要做的事, 是去判断一下一个 `pos` 是否越界, 是否合法. 所以我们先去掉这两个操作, 如果有必要, 增加一个 `pos` 合法性判断的操作. 这种概念上的讨论, 越早进行越好, 越到后续会有大量沉没成本.

然后我们这里没有考虑具体将数据存放在什么地方, 由于我们这里从学习角度出发, 放弃 C++ 已经存在的数据结构, 所以干脆直接从向系统申请内存开始. 这样我们需要一个指针来记录数据在内存中开始的地址. 将上述代码调整如下: 

In [2]:
class ArrayList
{
private:
    char* data;
public:
    void printList();
    void makeEmpty();
    int find(char x);
    void insert(char x, int pos);
    void remove(char x);
    char findKth(int pos);
};

在这里我们遗漏了两个必须要有的操作: 出生和死亡. 也即对任何一个类而言, 都需要考虑, 
+ 当一个实例被创建和初始化数据时；
+ 当一个实例被删除时；

我们需要做的事. 在 C++ 的规范中, 任何类都必须有以下两个成员操作:
+ **构造函数, constructor**. 函数名和类名一致, 没有返回类型. 参数可以自定. 这是在创建实例时, 会第一个调用的函数,因此可以在这里规定实例的数据如何初始化.
+ **析构函数, destructor**. 函数名为\~加类名, 同样没有返回类型, 也没有参数. 这是当一个实例正常删除时, 最后被调用的一个函数, 在这里可以做一些收尾工作, 比如确保动态分配的内存都已经安全释放.

如果没有显式声明构造和析构函数, 那么编译器也会默认产生一个缺省的构造函数和析构函数. 它们的都相当于一个带空参数的空函数. 构造函数可以重载, 但析构不会重载(为什么?).  

在 ArrayList 中, 我们需要在构造函数中申请数据内存, 而在析构函数中确保这部分数据被正确释放, 所以默认的构造和析构不一定合适. 我们做以下的设计: 

In [17]:
#include <iostream>

class ArrayList
{
private:
    // C++ 11 以后允许给类成员赋初指, 且发生在任何构造函数之前.
    char* data = nullptr;
public:
    void printList();
    void makeEmpty();
    int find(char x);
    void insert(char x, int pos);
    void remove(char x);
    char findKth(int pos);
    // 新增的构造函数, 这个也是缺省构造函数的实现方式
    // 但一旦有了自定义的构造函数, 那么缺省构造函数也必须显式实现
    ArrayList() {  }
    // 重载一个构造函数, 考虑一下这么设计是否合理? 
    ArrayList(int N) { data = new char[N]; };
    // 新增析构函数, 这里不能使用缺省析构函数
    ~ArrayList() { if (data != nullptr) delete [] data; }
};

这里我们实际上重载了构造函数, 也就是提供了两个不同版本的构造函数, 它们通过参数调用区别. 这也是析构函数为何不能重载的原因, 它没有参数. 尽管以上代码都是可行的, 但逻辑上这里引入了另一个问题: 我们如何确保数组大小是正确的. 这一点无法在 C++ 内部解决. 因为一旦动态内存分配了, 它就只是一个指针, 我们不能再通过任何语言内的机制去调查它的实际内存占用. 这是因为 C++ 在这一部分基本还是继承了 C 语言的机制, 而 C 语言作为一种直接面向硬件编程的语言, 有时效率比严谨更加重要(这一点现在越来越有疑问). 具体而言, 我们这里要让用户能合理得知数组大小的办法几乎只有增加一个属性, 但同时我们要完全依赖程序员确保这个表示大小的属性值总是对的. 如果程序员出错了, 没有任何客观机制中止这一点. 这就表明, 至少到现在为止, C++ 还不是一种完全客观安全的语言, 需要我们时刻注意类似内存泄漏等可能的隐患. C++ 确实在这些方面越来越注意. 比如通过 **const** 等限定词约束成员函数的行为. 

这里顺便也交待一下命名问题. 基本上只要意义明确即可, 尽量减少无意义的命名. 比如这里的 `pos` 是可以接受的, 而 `x` 和 `N` 则多少有点迷惑. 各函数的命名都用了驼峰风格, 基本上都是动词属性. 这些都供大家参考. 总之, 尽量形成一个你自己可以接受的, 比较合理且封闭的风格, 然后一以贯之就行. 

基于上述描述, 我们继续进化我们的 ArrayList.

In [18]:
#include <iostream>

class ArrayList
{
private:
    // C++ 11 以后允许给类成员赋初指, 且发生在任何构造函数之前.
    char* data = nullptr;
    // 用以表示数组大小, 必须靠程序员可靠维护.
    int size = 0;
public:
    // 这个 const 表示该成员函数不会修改类的任何数据.
    void printList() const;
    void makeEmpty();
    // 从实用角度, find 返回一个指针会更有意义, 所以这里也提供了动静两套版本.
    const char* find(char val) const ;  
    char* find(char val) ;
    // 原本的 find 保留, 但改名为 findIdx.
    int findIdx(char val) const;
    // 动态函数就不可以加 const 限制.
    void insert(char val, int pos);
    void remove(char val);
    // 不允许修改返回的指针, 也不允许函数修改类成员数据. 
    const char* findKth(int pos) const;
    // 这是上一个函数的重载, 允许修改返回的指针, 也允许修改函数成员的数据.
    char* findKth(int pos);
    ArrayList() { }
    ArrayList(int N) { data = new char[N]; };
    ~ArrayList() { delete [] data; }
};

在这个版本中, 我们注意一种新的重载约定. 一般函数重载是通过函数参数决定的, 但在 C++ 中, 当你对一个`const`对象或者一个非`const`对象调用`fun()`函数时，编译器会根据调用对象的`const`情况来决定调用哪个版本的函数。相比上一个版本的 `findKth`, 只读的版本少了一次数据复制, 而可写的版本事实上提供了进一步修改找到的数据的可能性. 是否要提供这种可能性, 是设计者要考虑的问题. 这里可以考虑将处在同一地位的 `find` 也改造成这样的双接口形式. 而将原本的 `find` 改名为 `findIdx`.

对于以上成员函数的实现, 不是一个困难的任务, 这里只介绍一些要点: 

+ 不要出现同一功能的重复代码.

In [20]:
const char* ArrayList::find(char val) const  
{  
    for (int i = 0; i < size; ++i) {  
        if (data[i] == val)  
        {  
            return &data[i];  
        }  
    }  
    return nullptr;  
}  
  
char* ArrayList::find(char val)   
{  
   return const_cast<char*>(static_cast<const ArrayList&>(*this).find(val));  
}  

input_line_33:1:24: error: out-of-line definition of 'find' does not match any declaration in '__cling_N519::ArrayList'
const char* ArrayList::find(char val) const  
                       ^~~~
input_line_32:7:9: note: member declaration does not match because it is not const qualified
    int find(char x);
        ^          ~
input_line_33:3:25: error: use of undeclared identifier 'size'
    for (int i = 0; i < size; ++i) {  
                        ^
input_line_33:4:13: error: use of undeclared identifier 'data'
        if (data[i] == val)  
            ^
input_line_33:6:21: error: use of undeclared identifier 'data'
            return &data[i];  
                    ^
input_line_33:14:1: error: function definition is not allowed here
{  
^


Interpreter Error: 

这里别管在线编译器的错误. 这段代码体现了的原则是相同的代码永远不要重复出现. 这里静态和动态的版本逻辑上除了权限以外, 会完全一致, 但是将其中一个版本完全复制一遍给另一个版本用, 除了会留下数据冗余的隐患, 更麻烦的是在一些多线程等并行场合, 会发生无法预计的错误. 一个基本原则是, 动态版本去复用静态版本. 中间用合适的类型转换. 比如这里, `static_cast<const ArrayList&>(*this)` 先做了第一次转换, 将"自己", `*this` 用 `static_cast` 转换成一个 `const ArrayList&`. 然后我们就可以调用静态版本的 `find`, 得到的结果也是一个 `const char *`, 所以再用 `const_cast<char*>` 去掉它的 `const` 属性. 这样避免了代码重复, 同时, 相对而言, 静态版本更安全. 这里要注意, `const_cast` 能去掉一个 `const` 对象的只读属性, 但如果这个对象声明时就是只读的, 或干脆就是一个常量, 那么这么做就会导致错误. 我们这里因为同时实现了动静两套版本, 所以不存在这个风险. 

类似地, `findKth` 也有同样的考虑. 而且注意这里的异常处理. C / C++ 的程序员有责任将任何可能的错误, 如越界, 内存泄漏等异常在代码中排除. 新的 C++ 提供了更多的工具, 但我们先从最原始的方式凸显这一动作的必要性. 

In [21]:
const char* ArrayList::findKth(int pos) const
{
    if (pos < 0 || pos >= size)
    {
        std::cerr << "Error: Invalid position." << std::endl;
        exit(-1);
    }
    return data + pos;
}

char* ArrayList::findKth(int pos) 
{
    return const_cast<char*>(static_cast<const ArrayList&>(*this).findKth(pos));  
}

input_line_34:1:24: error: out-of-line definition of 'findKth' does not match any declaration in '__cling_N519::ArrayList'
const char* ArrayList::findKth(int pos) const
                       ^~~~~~~
input_line_32:10:10: note: member declaration does not match because it is not const qualified
    char findKth(int pos);
         ^              ~
input_line_34:3:27: error: use of undeclared identifier 'size'
    if (pos < 0 || pos >= size)
                          ^
input_line_34:8:12: error: use of undeclared identifier 'data'
    return data + pos;
           ^
input_line_34:12:1: error: function definition is not allowed here
{
^


Interpreter Error: 

一个数据结构的实现难点一般都集中在动态操作部分, 也就是 `insert` 和 `remove` 等操作. 数组实现的线性表本质复杂度是 $O(N)$, 这一点无法改变, 但一些细节调整, 比如考虑 `insert` 或 `remove` 有一定概率连续出现, 因此尽量减少重新申请内存的次数等等. 但作为一个 C++ 入门级别的示例, 直观, 展示语法细节的需求高于非本质的效率提升, 因此我们先实现一个非常简单直接的版本, 而将改进放到后面去讨论(比如我们先不讨论 try/catch 机制和多线程处理, 也不考虑内存不够该怎么处理). 注意动态操作时, 和内存分配有关的异常检查是非常必要的. 

In [22]:
void ArrayList::insert(char val, int pos)
{
    if (pos < 0 || pos > size)
    {
        std::cerr << "Error: Invalid position." << std::endl;
        exit(-1);
    }
    if (size == 0)
    {
        if (data == nullptr)
            data = new char[1];
        else
        {
            std::cerr << "Warning: Allocating memory to a non-null pointer may result in memory leakage." 
            << std::endl;
            exit(-1);
        }
        data[0] = val;
        size++;
        return;
    }
    char* p = new char[size + 1];
    for (int i = 0; i < pos; i++)
        p[i] = data[i];
    p[pos] = val;
    for (int i = pos + 1; i < size + 1; i++)
        p[i] = data[i - 1];
    delete [] data;
    data = p;
    size++;
    return;
}

void ArrayList::remove(char val)
{
    int pos = findIdx(val);
    if (pos == -1)
    {
        std::cerr << "Error: The value is not in the list." << std::endl;
        exit(-1);
    }
    for (int i = pos + 1; i < size; i++)
        data[i - 1] = data[i];
    // 只改了 size, 没有释放内存.
    size--;
}

input_line_35:3:26: error: use of undeclared identifier 'size'
    if (pos < 0 || pos > size)
                         ^
input_line_35:8:9: error: use of undeclared identifier 'size'
    if (size == 0)
        ^
input_line_35:10:13: error: use of undeclared identifier 'data'
        if (data == nullptr)
            ^
input_line_35:11:13: error: use of undeclared identifier 'data'
            data = new char[1];
            ^
input_line_35:18:9: error: use of undeclared identifier 'data'
        data[0] = val;
        ^
input_line_35:19:9: error: use of undeclared identifier 'size'
        size++;
        ^
input_line_35:22:24: error: use of undeclared identifier 'size'
    char* p = new char[size + 1];
                       ^
input_line_35:24:16: error: use of undeclared identifier 'data'
        p[i] = data[i];
               ^
input_line_35:26:31: error: use of undeclared identifier 'size'
    for (int i = pos + 1; i < size + 1; i++)
                              ^
input_line_35:27:

Interpreter Error: 

现在我们考虑另一个在数据结构和类设计中常见的问题, 如何处理复制. 比如在实际工作中, 下面的代码是经常会用到的: 
```
ArrayList A(3), B;
ArrayList.insert(3, 0);
ArrayList.insert(2, 0);
ArrayList.insett(1, 0);

B = A;
```
这里我们先抛开一切, 最直观的要求就是希望完成这个赋值运算后, `B` 的内容要和 `A` 完全一样. 和缺省构造函数一样, 编译器默认也会提供一个缺省的赋值函数和一个被称为复制构造函数的东西, 后者专门完成下面这样的操作:
```
ArraryList C(A);
```
要求也是 `C` 的内容和 `A` 一样. 显然, 缺省提供复制和赋值, 在这个例子都不能完成我们的要求. 因为它们会把成员属性都对应复制一遍, 而在我们这里, `data` 是一个指针, 这样做的后果就是两个实例指向同一段内存数据. 这是万恶之源! 在类似动态内存分配的情况下, 我们需要手工设计这两个要求几乎一样的功能. 先扩展我们的类设计:

In [23]:
#include <iostream>

class ArrayList
{
private:
    // C++ 11 以后允许给类成员赋初指, 且发生在任何构造函数之前.
    char* data = nullptr;
    // 用以表示数组大小, 必须靠程序员可靠维护.
    int size = 0;
public:
    // 这个 const 表示该成员函数不会修改类的任何数据.
    void printList() const;
    void makeEmpty();
    // 从实用角度, find 返回一个指针会更有意义, 所以这里也提供了动静两套版本.
    const char* find(char val) const ;  
    char* find(char val) ;
    // 原本的 find 保留, 但改名为 findIdx.
    int findIdx(char val) const;
    // 动态函数就不可以加 const 限制.
    void insert(char val, int pos);
    void remove(char val);
    // 不允许修改返回的指针, 也不允许函数修改类成员数据. 
    const char* findKth(int pos) const;
    // 这是上一个函数的重载, 允许修改返回的指针, 也允许修改函数成员的数据.
    char* findKth(int pos);
    ArrayList() { }
    ArrayList(int N) { data = new char[N]; };
    ~ArrayList() { delete [] data; }
    // 复制构造函数, 这里对参数做了引用.
    ArrayList(const ArrayList &rhs);
    // 赋值运算的重载.
    ArrayList &operator=(const ArrayList &rhs);
};

观察一下这两段实现的区别, 复制构造函数, 因为是构造函数, 所以会有一个初始化过程, 因此我们不必对 `data` 和 `size` 做过多的检查, 它们会被赋初值. 而当赋值运算符调用时, `this` 所指的类实体, 也就是自身, 是处在一个正在使用中的状态, 因此必须做一些异常排除, 比如这里先过滤自己赋给自己的操作. 然后一个干脆的做法是先清空自己, 再接受新的数据. 这个做法也许未必效率最高, 但逻辑上最干净, 也最大程度避免了代码重复. 一个更极端的做法是连析构函数也调用 `makeEmpty()`, 但这样的话 `makeEmpty()` 对析构函数而言就多做了 `data = nullptr` 这件事. 一个解决办法是将清空内存这个过程单独的写成一个内部私有函数, 可以是 `inline`, 比如 `_clean()`, 然后大家都调用这个函数用于清空数据结构, 而后续的收尾工作各自去完成. 在一些更加复杂的结构中, 我们会考虑这么做. 而在 ArrayList 中, 显然没必要为 `delete [] data;` 构建一个函数. 

In [10]:
ArrayList::ArrayList(const ArrayList &rhs)
{
    size = rhs.size;
    if (size == 0)
    {
        data = nullptr;
        return;
    }
    data = new char[size];
    for (int i = 0; i < size; i++)
        data[i] = rhs.data[i];
    return;
}

ArrayList &ArrayList::operator=(const ArrayList &rhs)
{
    if (this != &rhs)
    {
        this->makeEmpty();
        size = rhs.size;
        data = new char[size];
        for (int i = 0; i < size; i++)
            data[i] = rhs.data[i];
    }
    return *this;
}

input_line_19:16:1: error: function definition is not allowed here
{
^


Interpreter Error: 

现在回头讨论一个我们在之前设计中没有考虑的因素: 函数的参数复制. 当形参将数据传递给函数内部时, 标准的做法会导致一次数据复制. 这个复制在函数调用的时刻发生. 而函数如果有返回值, 那么在 `return` 语句位置也会发生一次返回值的复制. C 语言和早期的 C++ 一般通过全局变量, 局部静态变量或者指针调用来避免这个复制, 提高函数调用的效率. 但这么做的后果是函数的出入接口会变得很复杂且失去一致性. 后续 C++ 引入了**引用**的概念, 即 `&` 放在变量名之前, 如果这个变量名在赋值号 `=` 的右侧, 比如:
```
double a = 10;
double *p = &a;
```
那么表示取 `a` 这个变量的地址. 但是如果出现在赋值号 `=` 的左侧, 比如:
```
double a = 10;
double &p = a;
```
那么表示对变量 `a` 的引用, 也就是说 `p` 就是 `a` 的另一个名字, 操作 `p` 就是操作 `a`.

将这个特性用于函数的参数和返回值传递, 就可以有效避免额外的数据复制. 同时还能保持接口的一致性. 为未来可能的抽象提供了基础. 大家注意到我们新引入的复制和赋值都采用了引用参数来减少参数复制的发生. 而赋值运算连返回值都变成了引用, 这么做的额外效果是, 如下面这样的"链式赋值"将被允许:
```
ArrayList A, B, C {'a'};
A = B = C;
```
这里的`A = B = C` 操作的具体实现相当与 `A = (B = C)`, 先执行 `B = C`, 将 `C` 的数据复制给 `B`, 这里的关键是表达式 `B = C` 返回的是什么? 回答 `return *this`, 也就是 `B` 的引用, 因此可以继续交给 `A`, 最终 `A`, `B` 和 `C` 是三个数据内容值相同, 但内存完全独立的对象. 这是我们要实现的目的. 这个过程有时被称为深拷贝逻辑. 但如果返回值不采用引用, 将赋值运算改为:
```
ArrayList ArrayList::operator=(const ArrayList &rhs)
```
那么执行完 `B = C` 以后, 返回的是一个 `ArrayList` 对象, 这个对象就是 `B` 么? 其实不是, 因为在 `return *this` 时, 如果需要接收的不是一个引用, 那么就会发生一次复制, 直接调用复制构造函数, 产生一个数据内容和 `*this` 一样的临时对象, 然后把这个临时对象返回出去(**任何非引用的参数传入和函数返回, 都是一次额外的复制**). 但因为它是临时对象, 函数结束以后, 它的生命周期就到头了, 实际上多调了一次额外的复制. 如果复制写的不够好, 还可能出现指向一个已经被销毁对象的可能, 这就是内存泄露. 最麻烦的是, 以上不论哪种实现, 语法上都是对的, 因次要靠程序员的自我修养来建议正确, 高效的实现方式. 

我们再检查一遍全部的 ArrayList 设计. 比如
```
int findIdx(char val) const;
```
这个函数, 目前形参 `val` 是发生额外的复制的. 如果未来 `char` 变成更大的数据结构, 那么这个复制可能代价昂贵. 避免的方式也很简单, 
```
int findIdx(const char &val) const;
```
这里 `&` 表示这是一个引用, 所以避免了参数调用时的数据复制. 但因为标准引用是具有完全权限的, 而 `find` 是一个 `const` 函数, 所以为了避免在 `findIdx` 内部改变 `val` 的值, 在引用前加 `const` 关键字, 限制它只能被读取. 

而
```
const char* find(char val) const ;  
char* find(char val) ;
```
这一对函数的引用改进就值得深入推敲. 首先返回指针本省并不存在额外数据复制的问题, 但是, 我们为什么要返回指针? 原始的设计动机如果就是为了操作其中的单个元素, 那么我们必须对返回值加 `*` 操作才能实现, 这就失去了数据接口的一致性. 除非我们就是要获得具体数据的内存地址(指针)完成一些特殊的目的, 比如调整内存分配. 否则, 仅为了操作单个数据, 完全可以用引用代替, 不但也能避免复制, 而且还能保持接口数据的一致性. 比如:
```
const char& find(const char &val) const ;  
char& find(char &val) ;
```
函数的返回值就是一个 `char` 类型, 直接可以读取, 甚至可以修改:
```
char A = 'a';
find(A) = 'b';
```
但使用引用返回也有一个潜在问题需要注意. 如果你的查找函数不能找到给定的值, 你需要决定怎么处理. 如果你返回一个指针, 你可以返回 `nullptr` 作为查找失败的标志. 然而, 引用需要绑定到一个对象上, 在函数返回时, 你必须返回一个有效的对象. 这可能需要你定义一个特殊的返回值来表示没有找到给定的值, 并需要使用者去检查这个特殊值. 比如:
```
static char invalidValue = -250;  
```
这样用户需要确认返回的字符是否等于 `invalidValue` 以确定查找是否成功, 这可能增加了使用的复杂性. 所以, 返回值类型为引用还是指针, 取决于具体的使用情况和设计考量, 没有绝对的优劣之分. 

沿用上述的原则, 我们将
```
void insert(char val, int pos);
```
改造成
```
void insert(const char &val, int pos);

```
这里对于 `pos` 参数, 在大多数可以预见的情况下, 它就是一个整数. 此时复制既不贵, 还能提高效率. 即便在考虑未来可能抽象类型的情况下, 一个数据结构的 `pos` 应该总是一个轻量级的设计, 比如一个指针等等. 我能考虑到的最复杂的 `pos` 需求, 可能就是对一个 `n` 维数组的坐标. 但如果你真的把这个数据当作 `pos` 传来传去, 可能在设计上已经出了不小的问题. 数学上的历史习惯和直观, 往往有针对人类阅读的成份, 而现代计算机从底层物理结构和原理和人类就完全不一致, 在面向对象设计时, 完全忽视这一点往往会导致效率低下. 当然这个也是要看情况的.

在继续讨论之前, 我们再引入一个 C++ 类数据初始化的机制, 增加一个新的构造函数
```
ArrayList::ArrayList(std::initializer_list<char> initList);
```
具体实现如下. 它的作用是让用户可以以大括号的形式初始化实例:
```
ArrayList A {'a', 'b', 'c'};
```

In [24]:
ArrayList::ArrayList(std::initializer_list<char> initList) {  
    size = initList.size();  
    data = new char[size];  
  
    int i = 0;  
    for (auto it = initList.begin(); it != initList.end(); it++, i++) {  
        data[i] = *it;  
    }  
}  

input_line_38:1:12: error: out-of-line definition of 'ArrayList' does not match any declaration in
      '__cling_N523::ArrayList'
ArrayList::ArrayList(std::initializer_list<char> initList) {  
           ^~~~~~~~~
input_line_38:2:21: error: no member named 'size' in 'std::initializer_list<char>'
    size = initList.size();  
           ~~~~~~~~ ^
input_line_38:6:29: error: no member named 'begin' in 'std::initializer_list<char>'
    for (auto it = initList.begin(); it != initList.end(); it++, i++) {  
                   ~~~~~~~~ ^
input_line_38:6:53: error: no member named 'end' in 'std::initializer_list<char>'
    for (auto it = initList.begin(); it != initList.end(); it++, i++) {  
                                           ~~~~~~~~ ^


Interpreter Error: 

这里出现了一些新的语法特性. 
+ `std::initializer_list<char>`, 这是一个**模板类**, 具体什么是模板类, 我们之后再细说. 这里大家只需要知道这其实就是机器内置的一个 `list`, 专门接受用户提供的 `{'a', 'b', 'c'}` 这样的初始化信息. 它类型不定, 由用户在 `<char>` 中指定, 如果你的 `list` 是 `int`, 这里也可以用 `<int>`. 所以模板大家可以暂时认为是一种抽象了具体数据类型的类. 
+ 这个内置的 `list` 的设计目的就是在用户提供的数据不定长时, 快速读取数据列表. 所以它本质上只提供了两个关键信息, 
    + 数据在内存中的起始位置, 
    + 数据在内存中的结束位置. 

所以对它可以不用担心参数复制问题, 因为复制本身是非常廉价的, 本质上就复制了两个指针. 
+ 在内部读取时, 设计了一套 `iterator`, 也就是
```
for (auto it = initList.begin(); it != initList.end(); it++, i++) 
```
这个遍历, C++ 非常推崇用这种抽象**遍历器(iterator)** 的形式做遍历, 因为它本质上抽象了数据遍历的起点(`.begin()`)和终点(`.end()`), 从根本上避免了数据越界的问题. 相比我们之前的底层设计中
```
if (pos < 0 || pos > size)
{
    std::cerr << "Error: Invalid position." << std::endl;
    exit(-1);
}
```
这么做显得更加优雅. 
+ C++ 11 重新定义了 `auto` 关键字, 它的作用是自动推断类型. 在一个遍历过程中, 具体指针是什么类型是蕴含在遍历过程之中的(由 `iterator` 的来源决定), 因此在这里自动推断 `it` 的类型, 既突出了逻辑重点, 也避免了冗余错误, 比如给 `it` 一个错误的类型. 

遍历器或迭代器的设计理念, 我们后续会继续介绍. 这里大家可以把它看作一个特殊的指针, 用 `++it` 可以进行步进操作, 用 `*it` 可以进行内容操作. 以上是为了清晰表现 C++ 基础特性的代码, 在最新的工作中, 一种更加推荐的实现方式实际上是:

In [25]:
ArrayList::ArrayList(std::initializer_list<char> initList) {    
    size = initList.size();    
    data = new char[size];    
    
    int i = 0;    
    for (const auto& value : initList) {    
        data[i++] = value;    
    }    
}    

input_line_39:1:12: error: out-of-line definition of 'ArrayList' does not match any declaration in
      '__cling_N523::ArrayList'
ArrayList::ArrayList(std::initializer_list<char> initList) {    
           ^~~~~~~~~
input_line_39:2:21: error: no member named 'size' in 'std::initializer_list<char>'
    size = initList.size();    
           ~~~~~~~~ ^
In file included from input_line_5:1:
In file included from /home/hywang/anaconda3/envs/teaching/include/xeus/xinterpreter.hpp:13:
In file included from /home/hywang/anaconda3/envs/teaching/bin/../lib/gcc/../../x86_64-conda-linux-gnu/include/c++/9.4.0/functional:54:
In file included from /home/hywang/anaconda3/envs/teaching/bin/../lib/gcc/../../x86_64-conda-linux-gnu/include/c++/9.4.0/tuple:38:
In file included from /home/hywang/anaconda3/envs/teaching/bin/../lib/gcc/../../x86_64-conda-linux-gnu/include/c++/9.4.0/utility:76:
/home/hywang/anaconda3/envs/teaching/bin/../lib/gcc/../../x86_64-conda-linux-gnu/include/c++/9.4.0/initializer_list

Interpreter Error: 

因为遍历逻辑是非常频繁使用的, 因此这样的一种自动简约是非常合适的. 程序员不用在纠结到底遍历的是什么, 同时也不会再有犯错的可能. 这种设计风格是一种大趋势.

到这里, 以下的情况我们都已经能够处理:
```
// initializer_list 初始化, 更推荐
ArrayList A {'a', 'b', 'c'};
// 复制构造
ArrayList B = A;
// 和第一种等价
ArrayList C = {'d', 'e', 'd'};
ArrayList D;
// 复制构造
ArrayList E(C);
// 赋值运算
D = A;
```

但有一种实际工作中可能出现的情形, 不包含在上述情况内:
```
ArrayList A;
A = {'a', 'b', 'c'};
```
因为右端是一个 `initializer_list`, 它只是一段数据, 不是一个类. 要扩展这种功能, 需要提供一个新的赋值运算接口:
```
ArrayList& operator=(std::initializer_list<char> list);
```
以及相应的实现. 这一点不难. 然而下面这种情况呢?
```
A = ArrayList {'a', 'b', 'c'};
```
首先这段代码是否合法? 在目前的设计中, 右端会产生一个临时的 `ArrayList` 对象, 内容指定. 然后复制给 `A`. 所以是合法的. 但这么做无法避免一重额外的数据复制. 这次的复制是显式的, 就是赋值运算的功能. 如果要避免这一层复制, 我们可以考虑采用"右值引用"的策略. 这是一个 C++ 11 引入的新概念. 传统的**左值(Lvalue)** 和**右值(Rvalue)**, 分别代表可以出现在赋值运算符 `=` 左边的量, 如一个变量; 以及**只能**出现在赋值运算符右边的量, 比如一个常量, 或者混有常量的表达式, 等等. 但 C++ 11 进一步区分了右值, 将它分成**纯右值(PRvalue)**, 即传统意义的右值, 和 **将亡值(XRvalue)**, 即一个即将被销毁的对象, 典型的例子就是上面代码中的 `A = ArrayList {'a', 'b', 'c'}` 中的右值部分, 这是一个临时对象, 它即将在这句赋值语句之后被销毁. 现在 C++ 11 提供了被称为 **右值引用** 的策略, 允许将这个即将被销毁的对象中的全部或部分数据**转移**出去, 交给另一个对象控制. 这个临时对象仍然会在这句语句之后被销毁, 但是它的部分数据控制权因为被转移, 所以得以保留. 下面是一个具体实现的例子:

In [12]:
class ArrayList {  
public:  
    // ...其它代码...  
      
    ArrayList& operator=(ArrayList&& other) noexcept {  
        if (this != &other) {  
            // 添加清理现有数据的代码
            this->makeEmpty();
            // 移动other的数据到当前实例  
            size = other.size;
            data = other.data;
            // 注意在移动后, other可能处在一个"未定义"但是可析构的状态, 
            // 所以要把原数据指针位置设成 nullptr, 以防在析构时被销毁.
            other.data = nullptr;
        }  
        return *this;  
    }  
};  

input_line_21:8:19: error: no member named 'makeEmpty' in 'ArrayList'
            this->makeEmpty();
            ~~~~  ^
input_line_21:10:13: error: use of undeclared identifier 'size'
            size = other.size;
            ^
input_line_21:10:26: error: no member named 'size' in 'ArrayList'
            size = other.size;
                   ~~~~~ ^
input_line_21:11:13: error: use of undeclared identifier 'data'
            data = other.data;
            ^
input_line_21:11:26: error: no member named 'data' in 'ArrayList'
            data = other.data;
                   ~~~~~ ^
input_line_21:14:19: error: no member named 'data' in 'ArrayList'
            other.data = nullptr;
            ~~~~~ ^


Interpreter Error: 

这里 `noexcept` 是一个 C++ 11 引入的新关键字, 意思是绝对不会出现异常. 或者说, 一旦有异常出现, 就直接终止全部运行. 一般在构建转移赋值运算, 我们会考虑用这个关键字以配合 C++ 其他标准提高效率. 这样的赋值运算又被称为 **移动赋值运算(Move Assignment Operator)**, 类似还有 **移动构造函数(Move Constructor)**, 为了设计完整性, 我们先提供它的一个设计
```
ArrayList(ArrayList&& other) noexcept;
```

这里对于移动赋值运算的实现, 有一个非常纠结的点, 你必须先确保左边的对象清空数据, 然后结束之后还要确保右边数据已经解除控制. 如果能够直接把左右两边的控制权交换一下, 那么就非常自然实现了我们的目标: 左边对象获得了右边数据的控制权, 然后左边对象的控制权交给了右边, 当右边这个临时对象被销毁时, 原始左边的数据也自然一起被销毁了. 这么做不但逻辑清晰, 而且从根本上避免了内存泄露的可能. 最好的异常处理就是从根本逻辑上**避免异常的可能性**. 

In [26]:
ArrayList& operator=(ArrayList&& other) noexcept {    
    if (this != &other) {    
        std::swap(data, other.data);  
        std::swap(size, other.size);  
    }    
        return *this;    
}   

input_line_40:2:51: error: function definition is not allowed here
 ArrayList& operator=(ArrayList&& other) noexcept {    
                                                  ^


Interpreter Error: 

这里 `std::swap` 也是 algorithm 头文件提供的, 就是一个专门用于交换的过程. 它同样不纠结交换的是什么具体类型, 只是确保交换的逻辑清晰和高效. 同时我们这里 `noexcept` 也加的格外理直气壮. 大家可能已经注意到, 原始版本中, `makeEmpty()` 并不能从设计逻辑上避免异常.  

最后注意复制构造并不需要移动版本, 顾名思义, 这玩意本身的逻辑就是复制. 我们先把到目前为止的设计和实现都先对齐一次:

In [13]:
#include <iostream>
#include <algorithm>
#include <cstdlib>

class ArrayList
{
private:
    // C++ 11 以后允许给类成员赋初指, 且发生在任何构造函数之前.
    char* data = nullptr;
    // 用以表示数组大小, 必须靠程序员可靠维护.
    int size = 0;
public:
    // 这个 const 表示该成员函数不会修改类的任何数据.
    void printList() const;
    void makeEmpty();
    // 从实用角度, find 返回一个指针会更有意义, 所以这里也提供了动静两套版本.
    const char* find(const char &val) const ;  
    char* find(const char &val) ;
    // 原本的 find 保留, 但改名为 findIdx.
    int findIdx(const char &val) const;
    // 动态函数就不可以加 const 限制.
    void insert(const char &val, int pos);
    void remove(char &val);
    // 不允许修改返回的指针, 也不允许函数修改类成员数据. 
    const char* findKth(int pos) const;
    // 这是上一个函数的重载, 允许修改返回的指针, 也允许修改函数成员的数据.
    char* findKth(int pos);
    ArrayList() { }
    // 有变化.
    ArrayList(int N, const char& val); 
    ~ArrayList() { if (data != nullptr) delete [] data; }
    // 复制构造函数, 这里对参数做了引用.
    ArrayList(const ArrayList &rhs);
    // 赋值运算的重载.
    ArrayList &operator=(const ArrayList &rhs);
    // 列表初始化.
    ArrayList(std::initializer_list<char> initList);
    // 移动赋值.
    ArrayList& operator=(ArrayList&& other) noexcept;
    // 移送构造; 
    ArrayList(ArrayList&& other) noexcept;
};

注意原本的设计其实有一个逻辑隐患, 就是
```
ArrayList(int N) { data = new char[N]; };
```
这个构造函数, 提供了 `N` 个元素的内存空间, 但它并没有给出这 `N` 个数的值, 也没有调整 `size`. 那么如果调整 `size = N` 是否可以? 我们知道, 即便在我们之前的设计中, 实际内存大小和 `size` 并不一定总是相等的, 而且这样一来, 这凭空出现的 `N` 个元素的值也是混乱的. 一种做法是定 `size` 再定初值, 比如上面的设计就改成:

In [27]:
ArrayList(int N, const char& val) 
{
    if (N > 0) {  
        data = new char[N];  
        size = N;
        std::fill_n(data, N, val);  
    } else {  
        std::cerr << "The size of the list is invalid." << std::endl;
        exit(-1);
    }  
};

input_line_41:2:16: error: expected '(' for function-style cast or type construction
 ArrayList(int N, const char& val) 
           ~~~ ^
input_line_41:2:19: error: expected expression
 ArrayList(int N, const char& val) 
                  ^


Interpreter Error: 

这里 `std::fill_n` 就是转门处理这种情况的, 等价于一个遍历的赋值, 它也属于算法库 `<algorithm>`, 也是 C++ 11 后才引入的. 或者你专门再定一个属性, 如 `capacity` 专门用于记录实际内存大小, 同时别忘了把 `size` 置 0. 记录内存大小会增加所有动态操作的工作量, 但也提供了充分利用内存和尽量避免 `new` 和 `delete` 操作的空间, 在复杂的数据结构中, 也是经常被使用的设计手段.

上面的 ArrayList 的设计基本上已经是一份可以接受的教学示范代码(用于实际工作还效率不行). 我们最后讨论一下类型抽象来告一段落. 以上设计我们的元素本身都是 `char`, 但实际上, 我们注意到如果把这个元素类型改成一些基础类型, 如 `int`, `double`, 甚至是 `complex` 或是我们自己定义的类, 更有甚者, 可不可以元素也是 ArrayList? 从逻辑上这些当然都是可以的. 而实现上, 没有理由专门为一个特殊的类型再重复这些代码, 不但蠢, 而且严重违背了代码不复制的原则. 这里更加合理的做法就是数据类型抽象. 最朴素的做法, 可能是用上古的 C 语言提供的 `typedef` 或者宏定义, 如 
```
#define ELE_TYPE double
```
但在 C++ 中, 有更优雅的做法, **模板(template)**. 具体说, 就是把类写成这样:

In [14]:
#include <iostream>
#include <algorithm>
#include <cstdlib>

template <typename ELE_T>
class ArrayList
{
private:
    // C++ 11 以后允许给类成员赋初指, 且发生在任何构造函数之前.
    ELE_T* data = nullptr;
    // 用以表示数组大小, 必须靠程序员可靠维护.
    int size = 0;
public:
    // 这个 const 表示该成员函数不会修改类的任何数据.
    void printList() const;
    void makeEmpty();
    // 从实用角度, find 返回一个指针会更有意义, 所以这里也提供了动静两套版本.
    const ELE_T* find(const ELE_T &val) const ;  
    ELE_T* find(const ELE_T &val) ;
    // 原本的 find 保留, 但改名为 findIdx.
    int findIdx(const ELE_T &val) const;
    // 动态函数就不可以加 const 限制.
    void insert(const ELE_T &val, int pos);
    void remove(ELE_T &val);
    // 不允许修改返回的指针, 也不允许函数修改类成员数据. 
    const ELE_T* findKth(int pos) const;
    // 这是上一个函数的重载, 允许修改返回的指针, 也允许修改函数成员的数据.
    ELE_T* findKth(int pos);
    ArrayList() { }
    // 有变化.
    ArrayList(int N, const ELE_T& val); 
    ~ArrayList() { if (data != nullptr) delete [] data; }
    // 复制构造函数, 这里对参数做了引用.
    ArrayList(const ArrayList &rhs);
    // 赋值运算的重载.
    ArrayList &operator=(const ArrayList &rhs);
    // 列表初始化.
    ArrayList(std::initializer_list<ELE_T> initList);
    // 移动赋值.
    ArrayList& operator=(ArrayList&& other) noexcept;
    // 移送构造; 
    ArrayList(ArrayList&& other) noexcept;
    // 允许初始化赋值运算.
    ArrayList &operator=(std::initializer_list<char> list);
};

```
template <typename ELE_T>
class ArrayList
{ ... };
```
这个称为模板类, 这里抽象了数据类型 `ELE_T`, 我们可以用这个抽象数据类型来设计和实现我们的类, 而在用户使用时, 只需 `ArrayList<实际类型>` 就可以根据需要指定 `ELE_T` 的实际类型. 比如:
```
ArrayList<char> A; 
ArrayList<int> B;
ArrayList<ArrayList<char> > C;
```
模板类是一个生成类的指导模板, 编译器根据这个模板在编译的时候去产生实际的类. 这么做的一个可能的问题是, 由于类型是抽象的, 因此可以同样以模板化的形式构建类的具体实现, 但无法将函数实现预编译成二进制代码. 也就是说, 具体的实现过程会暴露给最终用户. 这对于开源软件而言不是一个问题, 而非开源的软件, 则需要用一些较为繁琐的技术手段来绕开这一点.

C++ 是一门有较深逻辑内涵的高级语言, 学习并不容易. 但目前我们已经介绍的部分, 足以先应付接下去具体算法和数据结构内容展开, 更多的C++特性我们将结合具体的内容讨论. 这也是 C++ 学习的一个关键: 你永远也不可能只看语法书学会 C++, 你必须主动去创造点什么, 然后改进你对 C++ 的领悟和技巧. 我们总结一下上面所有内容, 提供一个阶段性版本:

In [28]:
#include <iostream>
#include <algorithm>
#include <cstdlib>

template <typename ELE_T>
class ArrayList
{
private:
    // C++ 11 以后允许给类成员赋初指, 且发生在任何构造函数之前.
    ELE_T* data = nullptr;
    // 用以表示数组大小, 必须靠程序员可靠维护.
    int size = 0;
    // 专门用于处理各种异常.
    enum ERR_CODE 
    {  
        SUCCESS = 0,  
        INVALID_POS = -1,  
        UNKNOW_POINTER = -2, 
        MISSING_POINTER = -3,
        OPETATING_EMPTY_LIST = -4,
        INVALID_SIZE = -5   
    };  
    void _goto_exception (ERR_CODE err_code) const;

public:
    // 这个 const 表示该成员函数不会修改类的任何数据.
    // 这个函数其实非常 C 风格, 但先遵重课本的展开方式.
    void printList() const;
    void makeEmpty();
    // 从实用角度, find 返回一个指针会更有意义, 所以这里也提供了动静两套版本.
    const ELE_T* find(const ELE_T &val) const ;  
    ELE_T* find(const ELE_T &val) ;
    // 原本的 find 保留, 但改名为 findIdx.
    int findIdx(const ELE_T &val) const;
    // 动态函数就不可以加 const 限制.
    void insert(const ELE_T &val, int pos);
    void remove(const ELE_T &val);
    // 不允许修改返回的指针, 也不允许函数修改类成员数据. 
    const ELE_T& findKth(int pos) const;
    // 这是上一个函数的重载, 允许修改返回的指针, 也允许修改函数成员的数据.
    ELE_T& findKth(int pos);
    ArrayList() { }
    // 有变化.
    ArrayList(int N, const ELE_T& val); 
    ~ArrayList() { if (data != nullptr) delete [] data; }
    // 复制构造函数, 这里对参数做了引用.
    ArrayList(const ArrayList &rhs);
    // 赋值运算的重载.
    ArrayList &operator=(const ArrayList &rhs);
    // 列表初始化.
    ArrayList(std::initializer_list<ELE_T> initList);
    // 移动赋值.
    ArrayList& operator=(ArrayList&& other) noexcept;
    // 移送构造; 
    ArrayList(ArrayList&& other) noexcept;
    // 允许初始化赋值运算.
    ArrayList &operator=(std::initializer_list<ELE_T> list);
};

template <typename ELE_T>
void ArrayList<ELE_T>::_goto_exception(ArrayList<ELE_T>::ERR_CODE error_code) const
{
    switch (error_code)
    {
    case ArrayList<ELE_T>::SUCCESS:
        // do nothing.
        break;
    case ArrayList<ELE_T>::INVALID_POS:
        std::cerr << "Error: Invalid position." << std::endl;
        exit(-1);
        break;
    case ArrayList<ELE_T>::UNKNOW_POINTER:
        std::cerr << "Error: An empty list has a non-null pointer." << std::endl;
        delete [] data;
        exit(-1);
        break;
    case ArrayList<ELE_T>::MISSING_POINTER:
        std::cerr << "Error: A non-empty list has a null pointer." 
                  << std::endl;
        exit(-1);
        break;
    case ArrayList<ELE_T>::OPETATING_EMPTY_LIST:
        std::cerr << "Warning: Try to find the k-th element in an empty list." << std::endl;
        // do noting.
        break;
    case ArrayList<ELE_T>::INVALID_SIZE:
        std::cerr << "The size of the list is invalid." << std::endl;
        exit(-1);
        /* code */
    
    default:
        std::cerr << "Error: Unknown error." << std::endl;
        exit(-1);
        break;
    }
}

template <typename ELE_T>
void ArrayList<ELE_T>::printList() const
{
    if (size == 0)
    {
        std::cout << "The list is empty." << std::endl;
        return;
    }
    for (int i = 0; i < size; i++)
        std::cout << data[i] << " ";
    std::cout << std::endl;
    // 作为void函数, 不需要 return 语句.
    // 这个可以作为个人风格的一种体现.
    return;
}

template <typename ELE_T>
void ArrayList<ELE_T>::makeEmpty()
{
    if (size == 0)
    {
        if (data != nullptr)
            _goto_exception(UNKNOW_POINTER);
    }
    else
    {
        size = 0;
        if (data != nullptr)
        {
            delete [] data;
            data = nullptr;
        }
        else
            _goto_exception(MISSING_POINTER);
    }
    return;
}

template <typename ELE_T>
const ELE_T* ArrayList<ELE_T>::find(const ELE_T &val) const  
{  
    for (int i = 0; i < size; ++i) {  
        if (data[i] == val)  
        {  
            return &data[i];  
        }  
    }  
    return nullptr;  
}

template <typename ELE_T>
ELE_T* ArrayList<ELE_T>::find(const ELE_T &val) 
{  
    return const_cast<ELE_T*>(static_cast<const ArrayList<ELE_T>&>(*this).find(val));
}

template <typename ELE_T>
void ArrayList<ELE_T>::insert(const ELE_T &val, int pos)
{
    // 对于动态内存分配时可能的异常, 一定要认真排除, 这是万恶之源. 
    if (pos < 0 || pos > size)
        _goto_exception(INVALID_POS);
    if (size == 0)
    {
        if (data == nullptr)
            data = new ELE_T[1];
        else
            _goto_exception(UNKNOW_POINTER);
        data[0] = val;
        size++;
        return;
    }
    if (data == nullptr)
        _goto_exception(MISSING_POINTER);
    ELE_T* p = new ELE_T[size + 1];
    for (int i = 0; i < pos; i++)
        p[i] = data[i];
    p[pos] = val;
    for (int i = pos + 1; i < size + 1; i++)
        p[i] = data[i - 1];
    delete [] data;
    data = p;
    size++;
    return;    
}

template <typename ELE_T>
const ELE_T& ArrayList<ELE_T>::findKth(int pos) const
{
    if (size == 0)
    {
        if (data == nullptr)
            _goto_exception(OPETATING_EMPTY_LIST);   
        else
            _goto_exception(UNKNOW_POINTER);
    }
    if (pos < 0 || pos >= size)
        _goto_exception(INVALID_POS);
    return data[pos];
}

template <typename ELE_T>
ELE_T& ArrayList<ELE_T>::findKth(int pos)
{
    return const_cast<ELE_T&>(static_cast<const ArrayList<ELE_T>&>(*this).findKth(pos));
}

template <typename ELE_T>
int ArrayList<ELE_T>::findIdx(const ELE_T &val) const
{
    const ELE_T* p = find(val);
    if (p == nullptr)
        return -1;
    else
        return p - data;
}

template <typename ELE_T>
void ArrayList<ELE_T>::remove(const ELE_T &val)
{
    int pos = findIdx(val);
    // 没有找到并不认为是一个错误, 直接返回. 
    if (pos == -1)
        return;
    for (int i = pos + 1; i < size; i++)
        data[i - 1] = data[i];
    // 只改了 size, 没有释放内存. 这样如果出现连续的删除操作, 就不会频繁的申请释放内存.
    // 同样的思路也可用于 insert 函数. 但那样的话就需要另一个成员变量 capacity, 
    // 专门用于管理实际内存使用量.
    size--;
}

template <typename ELE_T>
ArrayList<ELE_T>::ArrayList(int N, const ELE_T& val)
{
    if (N > 0) {  
        data = new ELE_T[N];  
        size = N;
        std::fill_n(data, N, val);  
    } else 
        _goto_exception(INVALID_SIZE);
}

template <typename ELE_T>
ArrayList<ELE_T>::ArrayList(const ArrayList<ELE_T> &rhs)
{
    if (rhs.size == 0)
    {
        data = nullptr;
        size = 0;
        return;
    }
    if (rhs.data == nullptr)
        _goto_exception(MISSING_POINTER);
    data = new ELE_T[rhs.size];
    size = rhs.size;
    for (int i = 0; i < size; i++)
        data[i] = rhs.data[i];
}

template <typename ELE_T>
ArrayList<ELE_T> & ArrayList<ELE_T>::operator=(const ArrayList &rhs)
{
    if (this != &rhs)
    {
        this->makeEmpty();
        size = rhs.size;
        data = new ELE_T[size];
        for (int i = 0; i < size; i++)
            data[i] = rhs.data[i];
    }
    return *this;
}

template <typename ELE_T>
ArrayList<ELE_T> & ArrayList<ELE_T>::operator=(std::initializer_list<ELE_T> list)
{
    this->makeEmpty();
    size = list.size();
    data = new ELE_T[size];
    int i = 0;
    for (auto &ele : list)
        data[i++] = ele;
    return *this;
}

template <typename ELE_T>
ArrayList<ELE_T>& ArrayList<ELE_T>::operator=(ArrayList&& other) noexcept
{
    if (this != &other)
    {
        std::swap(data, other.data);  
        std::swap(size, other.size);     
    }
    return *this;
}

template <typename ELE_T>
ArrayList<ELE_T>::ArrayList(std::initializer_list<ELE_T> initList)
{
    size = initList.size();
    data = new ELE_T[size];
    int i = 0;
    for (auto &ele : initList)
        data[i++] = ele;
}

template <typename ELE_T>
ArrayList<ELE_T>::ArrayList(ArrayList&& other) noexcept
{
    std::swap(data, other.data);  
    std::swap(size, other.size);      
}

注意一些要点:
+ 统一用一个 `_goto_exception` 函数来处理全部的异常, 这也是代码不重复的原则的体现. 然后用枚举来使我们的错误代码更加清晰. 
+ 这份代码依旧是教学型的, 没有考虑连续动态操作的效率提升, 也没有考虑标准的异常处理 try/catch, 以及多线程情形.